# Unsupervised Domain Adaptation Project


## 1: Data download
Load data to project from Google Drive. Copy a subset of classes of images to the path:
- `adaptiope_small/product_images`
- `adaptiope_small/real_life` 

two directories. They represent images from two different domain **product** and **real_life**

In [3]:
from os import makedirs, listdir
from tqdm import tqdm
from google.colab import drive
from os.path import join
from shutil import copytree

drive.mount('/content/gdrive')

!mkdir dataset
!cp "gdrive/My Drive/Colab Notebooks/data/Adaptiope.zip" dataset/
# !ls dataset

!unzip -qq dataset/Adaptiope.zip   # unzip file

!rm -rf dataset/Adaptiope.zip 
!rm -rf adaptiope_small

Mounted at /content/gdrive


In [4]:
!mkdir adaptiope_small
classes = listdir("Adaptiope/product_images")
print(classes)
classes = ["backpack", "bookcase", "car jack", "comb", "crown", "file cabinet", "flat iron", "game controller", "glasses",
           "helicopter", "ice skates", "letter tray", "monitor", "mug", "network switch", "over-ear headphones", "pen",
           "purse", "stand mixer", "stroller"]
domain_classes = ["product_images", "real_life"]
for d, td in zip(["Adaptiope/product_images", "Adaptiope/real_life"], ["adaptiope_small/product_images", "adaptiope_small/real_life"]):
  makedirs(td)
  for c in tqdm(classes):
    c_path = join(d, c)
    c_target = join(td, c)
    copytree(c_path, c_target)

['hoverboard', 'wristwatch', 'stethoscope', 'knife', 'screwdriver', 'desk lamp', 'game controller', 'purse', 'ruler', 'bookcase', 'grill', 'telescope', 'fighter jet', 'keyboard', 'magic lamp', 'shower head', 'coat hanger', 'network switch', 'microwave', 'axe', 'rubber boat', 'boxing gloves', 'toothbrush', 'bicycle helmet', 'acoustic guitar', 'stroller', 'scissors', 'skateboard', 'tape dispenser', 'hard-wired fixed phone', 'in-ear headphones', 'ice cube tray', 'letter tray', 'tank', 'chainsaw', 'electric shaver', 'cellphone', 'vr goggles', 'tent', 'stand mixer', 'handgun', 'fan', 'corkscrew', 'power drill', 'stapler', 'power strip', 'puncher', 'scooter', 'usb stick', 'speakers', 'printer', 'crown', 'laptop', 'sewing machine', 'backpack', 'vacuum cleaner', 'smoking pipe', 'bicycle', 'pikachu', 'binoculars', 'lawn mower', 'calculator', 'watering can', 'glasses', 'drum set', 'nail clipper', 'wallet', 'baseball bat', 'monitor', 'rifle', 'fire extinguisher', 'spatula', 'snow shovel', 'over-e

100%|██████████| 20/20 [00:00<00:00, 25.34it/s]


In [5]:
product_path = 'adaptiope_small/product_images'
real_life_path = 'adaptiope_small/real_life'

## 2: Domain-Adversarial training of Neural Network
We implement DANN UDA method [DANN](https://arxiv.org/pdf/1505.07818.pdf)

 

### 2.0: Import Libraries and Data Loading


In [6]:
from PIL import Image
from os.path import join
import math

img = Image.open(join(product_path, 'backpack', 'backpack_003.jpg'))
print('Image size: ', img.size)
#img

Image size:  (679, 679)


import libraries

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import softmax
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.models import vgg11, alexnet 
from torch.utils.data import DataLoader, random_split
from torchvision.transforms.transforms import ToTensor

configuration constants

In [8]:
img_size = 256
# mean, std used by pre-trained models from PyTorch
mean, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
config = dict(epochs=10, batch_size=64,lr=0.01, wd=0.001, momentum=0.9, alpha=10, beta=0.75, gamma=10)

Configue GPU

In [9]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [10]:
def get_dataset(root_path):
  '''
    Get dataset from specific data path

    # parameters:
        root_path: path to image folder

    # return: train_loader, test_loader
  '''
  # Construct image transform
  image_transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.CenterCrop(img_size),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
  ])

  # Load data from filesystem
  image_dataset = ImageFolder(root_path, transform=image_transform)

  return image_dataset

def get_dataloader(dataset, batch_size, shuffle_train=True, shuffle_test=False):
  '''
    Get DataLoader from specific data path

    # parameters:
        dataset: ImageFolder instance
        batch_size: batch_size for DataLoader
        shuffle_train: whether to shuffle training data
        shuffle_test: whether to shuffle test data
  '''
  # Get train, test number
  num_total = len(dataset)
  num_train = int(num_total * 0.8 + 1)
  num_test  = num_total - num_train

  # random split dataset
  data_train, data_test = random_split(dataset, [num_train, num_test])

  # initialize dataloaders
  loader_train = DataLoader(data_train, batch_size=batch_size, shuffle=shuffle_train)
  loader_test  = DataLoader(data_test, batch_size=batch_size, shuffle=shuffle_test)

  return loader_train, loader_test

### 2.1 Define Feature Extractor with Pretrain Network

In [11]:
class FeatureExtractor(nn.Module):
  def __init__(self):
    super(FeatureExtractor, self).__init__()

    # Feature Extractor with AlexNet
    self.feature_extractor = alexnet(weights='DEFAULT')
    self.feature_dim = self.feature_extractor.classifier[-1].in_features

    # make the last layer identity
    self.feature_extractor.classifier[-1] = nn.Identity()

  def forward(self, x):
    return self.feature_extractor(x)
  
  def output_dim(self):
    return self.feature_dim

### 2.2 Define Classifier, Discriminator with RevereLayerF for training the Feature Extractor

In [12]:
from torch.nn.modules.activation import Softmax
class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Classifier, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, output_dim),
            nn.LogSoftmax(dim=1)
        )
    
    def forward(self, X):
        return self.classifier(X) 

In [13]:
from torch.autograd import Function

class ReverseLayerF(Function):
    @staticmethod
    def forward(ctx, tensor):
        return tensor.view_as(tensor)

    @staticmethod
    def backward(ctx, grad_output):
        return grad_output.neg(), None

In [14]:
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.discriminator =  nn.Sequential(
            nn.Linear(int(input_dim), 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024,1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Sigmoid()
        )

    def forward(self, x):
        validity = self.discriminator(x)
        return validity 

In [15]:
class DANN(nn.Module):
  # def __init__(self, num_classes, adversarial=True):
  def __init__(self, num_classes):
    super(DANN, self).__init__()
    self.output_dim = num_classes

    # define inner network component
    self.feature_extractor = FeatureExtractor()
    self.classifier = Classifier(self.feature_extractor.output_dim(), num_classes)
    self.discriminator = Discriminator(self.feature_extractor.output_dim())  
  
  def forward(self, x):
    feature_output = self.feature_extractor(x)

    class_pred = self.classifier(feature_output)

    # Add a ReverseLayer here for negative gradient computation
    reverse_feature = ReverseLayerF.apply(feature_output)
    domain_pred = self.discriminator(reverse_feature)

    return class_pred, domain_pred 

### 2.3 Cost function

In [16]:
class BinaryDiceLoss(nn.Module):
    """Dice loss of binary class
    Args:
        smooth: A float number to smooth loss, and avoid NaN error, default: 1
        p: Denominator value: \sum{x^p} + \sum{y^p}, default: 2
        predict: A tensor of shape [N, *]
        target: A tensor of shape same with predict
        reduction: Reduction method to apply, return mean over batch if 'mean',
            return sum if 'sum', return a tensor of shape [N,] if 'none'
    Returns:
        Loss tensor according to arg reduction
    Raise:
        Exception if unexpected reduction
    """
    def __init__(self, smooth=1, p=2, reduction='mean'):
        super(BinaryDiceLoss, self).__init__()
        self.smooth = smooth
        self.p = p
        self.reduction = reduction

    def forward(self, predict, target):
        assert predict.shape[0] == target.shape[0], "predict & target batch size don't match"
        predict = predict.contiguous().view(predict.shape[0], -1)
        target = target.contiguous().view(target.shape[0], -1)

        num = torch.sum(torch.mul(predict, target), dim=1) + self.smooth
        den = torch.sum(predict.pow(self.p) + target.pow(self.p), dim=1) + self.smooth

        loss = 1 - num / den

        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        elif self.reduction == 'none':
            return loss
        else:
            raise Exception('Unexpected reduction {}'.format(self.reduction))

In [17]:
import torch.nn.functional as F
class DiceLoss(nn.Module):
    """Dice loss, need one hot encode input
    Args:
        weight: An array of shape [num_classes,]
        ignore_index: class index to ignore
        predict: A tensor of shape [N, C, *]
        target: A tensor of same shape with predict
        other args pass to BinaryDiceLoss
    Return:
        same as BinaryDiceLoss
    """
    def __init__(self, weight=None, ignore_index=None, **kwargs):
        super(DiceLoss, self).__init__()
        self.kwargs = kwargs
        self.weight = weight
        self.ignore_index = ignore_index

    def forward(self, predict, target):
        # one hot encode input
        num_class = predict.shape[1]
        # one hot
        target = F.one_hot(target, num_classes=num_class)
        
        assert predict.shape == target.shape, 'predict & target shape do not match'
        dice = BinaryDiceLoss(**self.kwargs)
        total_loss = 0
        predict = F.softmax(predict, dim=1)

        for i in range(target.shape[1]):
            if i != self.ignore_index:
                dice_loss = dice(predict[:, i], target[:, i])
                if self.weight is not None:
                    assert self.weight.shape[0] == target.shape[1], \
                        'Expect weight shape [{}], get[{}]'.format(target.shape[1], self.weight.shape[0])
                    dice_loss *= self.weights[i]
                total_loss += dice_loss

        return total_loss/target.shape[1]

In [18]:
def get_class_loss_func(dice_loss=False):
  if dice_loss:
    return DiceLoss()
  else:
    return nn.CrossEntropyLoss()

### 2.4 Optimizer

Setting the **learning rate** according to the original [paper](https://arxiv.org/pdf/1505.07818.pdf) section 5.2.2

$$ \mu_p =  \frac{\mu_0}{(1+\alpha \cdot p)^\beta}$$

where p is the training progress linearly changing from 0 to 1.

In [19]:
def get_optimizer(model, config, progress, adversarial=True):
  '''
  Config Optimizer
  '''
  learning_rate = config['lr']
  learning_rate = learning_rate / ((1 + config['alpha']*progress)**config['beta'])

  weight_decay  = config['wd']
  momentum      = config['momentum']

  feature_ext   = model.get_submodule("feature_extractor")
  classifier    = model.get_submodule("classifier")
  discriminator = model.get_submodule("discriminator")

  pre_trained_weights = feature_ext.parameters()

  if adversarial:
    other_weights = list(classifier.parameters()) + list(discriminator.parameters())
  else:
    other_weights = list(classifier.parameters())

  # assign parameters to parameters
  optimizer = torch.optim.SGD([
    {'params': pre_trained_weights},
    {'params': other_weights, 'lr': learning_rate}
  ], lr= learning_rate/10, weight_decay=weight_decay, momentum=momentum)
  
  return optimizer

### 2.5 Training Loop and Testing Loop

In [20]:
def train_loop(dataloader, model, device, progress, dice_loss=False):
  """
    Return:
      @best_state: best performance model state parameters
      @best_loss: best performance loss
  """
  size = len(dataloader.dataset)
  loss_fn = get_class_loss_func()

  optimizer = get_optimizer(model, config, progress, adversarial=False)

  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)
    
    # compute prediction and loss
    class_pred, _ = model(X)

    # classification loss
    loss = loss_fn(class_pred, y) 
    curr_loss = loss.item()
    
    # backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 10 == 0:
      current = batch * len(X)
      print(f"## Meter ## current loss: {curr_loss:>7f} [{current:>5d}/{size:>5d}]")

In [21]:
def test_loop(dataloader, model, device, dice_loss=False):
  test_loss, correct = 0, 0
  loss_fn = get_class_loss_func()

  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      class_pred, _ = model(X)

      test_loss += loss_fn(class_pred, y).item()
      correct += (class_pred.argmax(1) == y).type(torch.float).sum().item()

  size = len(dataloader.dataset)
  num_batches = len(dataloader)

  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

  return test_loss, correct

### 2.6 Training Function

In [22]:
def training(model, train_dataloader, test_dataloader, config, device, dice_loss=False):
  epochs = config['epochs']
  # print(f"Learning_rate {config['lr']}, weight_decay {config['wd']}")

  for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n------------------")
    progress = epoch/epochs

    train_loop(train_dataloader, model, device, progress)
    # test_loss, _ = test_loop(test_dataloader, model, device)

  print("Done")

## 3 Training without using Domain Adaptation techniques

 ### 3.1 Product Domain -> Real Life

In [23]:
# Get dataloader
product_dataset   = get_dataset(product_path)
real_life_dataset = get_dataset(real_life_path)

#### 3.1.1 Training on Source Domain

In [24]:
train_dataloader, test_dataloader = get_dataloader(product_dataset, config['batch_size'])

model = DANN(len(product_dataset.classes)).to(device)

# Training
training(model, train_dataloader, test_dataloader, config, device)

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

Epoch 1
------------------
## Meter ## current loss: 2.969600 [    0/ 1601]
## Meter ## current loss: 2.374293 [  640/ 1601]
## Meter ## current loss: 0.667915 [ 1280/ 1601]
Epoch 2
------------------
## Meter ## current loss: 3.042634 [    0/ 1601]
## Meter ## current loss: 0.791877 [  640/ 1601]
## Meter ## current loss: 0.303730 [ 1280/ 1601]
Epoch 3
------------------
## Meter ## current loss: 0.542020 [    0/ 1601]
## Meter ## current loss: 0.290702 [  640/ 1601]
## Meter ## current loss: 0.202121 [ 1280/ 1601]
Epoch 4
------------------
## Meter ## current loss: 0.236690 [    0/ 1601]
## Meter ## current loss: 0.092328 [  640/ 1601]
## Meter ## current loss: 0.135721 [ 1280/ 1601]
Epoch 5
------------------
## Meter ## current loss: 0.116931 [    0/ 1601]
## Meter ## current loss: 0.149550 [  640/ 1601]
## Meter ## current loss: 0.129107 [ 1280/ 1601]
Epoch 6
------------------
## Meter ## current loss: 0.068333 [    0/ 1601]
## Meter ## current loss: 0.041600 [  640/ 1601]
## Me

#### 3.1.2 Test on Real Life

In [25]:
loader_target_dataset = DataLoader(real_life_dataset, batch_size=config['batch_size'], shuffle=False)

# model.load_state_dict(torch.load('model_state.pt', map_location='cpu'))
test_loop(loader_target_dataset, model, device)

Test Error: 
 Accuracy: 63.7%, Avg loss: 1.228601 



(1.2286008251830935, 0.637)

In [26]:
del train_dataloader, test_dataloader, loader_target_dataset
del model
print(torch.cuda.memory_allocated())

513480192


### 3.2 Real Life -> Product


#### 3.2.1 Training on Real Life

In [27]:
train_dataloader, test_dataloader = get_dataloader(real_life_dataset, config['batch_size'])

model = DANN(len(real_life_dataset.classes)).to(device)

# Training
training(model, train_dataloader, test_dataloader, config, device)

Epoch 1
------------------
## Meter ## current loss: 3.015364 [    0/ 1601]
## Meter ## current loss: 2.728484 [  640/ 1601]
## Meter ## current loss: 2.072116 [ 1280/ 1601]
Epoch 2
------------------
## Meter ## current loss: 1.250300 [    0/ 1601]
## Meter ## current loss: 1.229087 [  640/ 1601]
## Meter ## current loss: 0.744018 [ 1280/ 1601]
Epoch 3
------------------
## Meter ## current loss: 0.605536 [    0/ 1601]
## Meter ## current loss: 0.621318 [  640/ 1601]
## Meter ## current loss: 0.503447 [ 1280/ 1601]
Epoch 4
------------------
## Meter ## current loss: 0.306284 [    0/ 1601]
## Meter ## current loss: 0.410999 [  640/ 1601]
## Meter ## current loss: 0.346206 [ 1280/ 1601]
Epoch 5
------------------
## Meter ## current loss: 1.159203 [    0/ 1601]
## Meter ## current loss: 0.474070 [  640/ 1601]
## Meter ## current loss: 0.406240 [ 1280/ 1601]
Epoch 6
------------------
## Meter ## current loss: 0.267251 [    0/ 1601]
## Meter ## current loss: 0.210166 [  640/ 1601]
## Me

#### 3.2.2 Testing on Product


In [28]:
loader_target_dataset = DataLoader(product_dataset, batch_size=config['batch_size'], shuffle=False)

# model.load_state_dict(torch.load('model_state.pt', map_location='cpu'))
test_loop(loader_target_dataset, model, device)

Test Error: 
 Accuracy: 80.7%, Avg loss: 0.659917 



(0.6599167459935416, 0.8065)

In [29]:
del train_dataloader, test_dataloader, loader_target_dataset
del model
print(torch.cuda.memory_allocated())

513021440


Dice Training

In [30]:
train_dataloader, test_dataloader = get_dataloader(real_life_dataset, config['batch_size'])

model = DANN(len(real_life_dataset.classes)).to(device)
# Training
dice_config = dict(epochs=10, batch_size=256, lr=0.1, wd=0.001, momentum=0.9, alpha=10, beta=0.75, gamma=10)
training(model, train_dataloader, test_dataloader, dice_config, device, dice_loss=True)

Epoch 1
------------------
## Meter ## current loss: 3.010717 [    0/ 1601]
## Meter ## current loss: 2.743840 [  640/ 1601]
## Meter ## current loss: 1.994464 [ 1280/ 1601]
Epoch 2
------------------
## Meter ## current loss: 0.983435 [    0/ 1601]
## Meter ## current loss: 0.805589 [  640/ 1601]
## Meter ## current loss: 0.866838 [ 1280/ 1601]
Epoch 3
------------------
## Meter ## current loss: 0.472337 [    0/ 1601]
## Meter ## current loss: 0.463499 [  640/ 1601]
## Meter ## current loss: 0.413025 [ 1280/ 1601]
Epoch 4
------------------
## Meter ## current loss: 0.353862 [    0/ 1601]
## Meter ## current loss: 0.447898 [  640/ 1601]
## Meter ## current loss: 0.251958 [ 1280/ 1601]
Epoch 5
------------------
## Meter ## current loss: 0.228203 [    0/ 1601]
## Meter ## current loss: 0.222434 [  640/ 1601]
## Meter ## current loss: 0.272569 [ 1280/ 1601]
Epoch 6
------------------
## Meter ## current loss: 0.294113 [    0/ 1601]
## Meter ## current loss: 0.201670 [  640/ 1601]
## Me

## 4: Define UDA functions


### 4.1 Adversarial Discriminator Loss

In [31]:
def get_discriminator_loss(source_pred, target_pred): 
    domain_pred = torch.cat((source_pred, target_pred),dim=0).cuda()
    #print(domain_pred.shape) # [128,1024]
    source_truth = torch.zeros(len(source_pred))
    target_truth = torch.ones(len(target_pred))
    domain_truth = torch.cat((source_truth, target_truth),dim=0).cuda()
    #print(domain_truth.shape) # [128]

    domain_loss = domain_truth*torch.log(1/domain_pred)+(1-domain_truth)*torch.log(1/(1-domain_pred))
    domain_loss = domain_loss.mean()

    return domain_loss 

### 4.2 Adversarial optimizer

In [32]:
def get_adversarial_optimizer(model, config, progress, adversarial=True):
  '''
  Get Adversarial Optimizers
  '''
  lr, wd, momtm = config['lr'], config['wd'], config['momentum']
  lr = lr / ((1 + config['alpha']*progress)**config['beta'])

  feature_ext   = model.get_submodule("feature_extractor")
  classifier    = model.get_submodule("classifier")
  discriminator = model.get_submodule("discriminator")

  pre_trained_weights   = feature_ext.parameters()
  classifier_weights    = classifier.parameters()
  discriminator_weights = discriminator.parameters()

  feature_optim       = torch.optim.SGD([{'params': pre_trained_weights}],     lr=lr/10, weight_decay=wd, momentum=momtm)
  classifier_optim    = torch.optim.SGD([{'params': classifier_weights}],      lr=lr,    weight_decay=wd, momentum=momtm)
  discriminator_optim = torch.optim.SGD([{'params': discriminator_weights}],   lr=lr,    weight_decay=wd, momentum=momtm)
  
  return feature_optim, classifier_optim, discriminator_optim 

### 4.3 Adversarial Train Loop

Setting the **domain adaptation parameter** according to the original [paper](https://arxiv.org/pdf/1505.07818.pdf) section 5.2.2

$$ \lambda_p = \frac{2}{1 + exp(-\gamma \cdot p)} - 1 $$

where p is the training progress linearly changing from 0 to 1.

In [33]:
def adversarial_train_loop(source_loader, target_loader, model, config, progress, device):
  """
  return:
    @best_state
    @best_loss
  """
  size = len(source_loader.dataset)
  
  # cross entropy loss
  classification_loss = get_class_loss_func()

  # Get three optimizer
  feature_optim, class_optim, discriminator_optim = get_adversarial_optimizer(model, config, progress)

  # Target data loader iterator
  iter_target = iter(target_loader)

  domain_adapt = 2 / (1 + math.exp(-config['gamma']*progress)) - 1

  for batch, (X_source, y_source) in enumerate(source_loader):
    try:
      X_target, _ = next(iter_target)
    except:
      iter_target = iter(target_loader)
      X_target, _ = next(iter_target)  

    # Some internal bug return nested tesnor with size 1
    if len(X_source) < 64:
      continue

    X_source, y_source, X_target = X_source.to(device), y_source.to(device), X_target.to(device)

    class_pred_source, domain_pred_source = model(X_source)
    _,                 domain_pred_target = model(X_target)

    class_loss   = classification_loss(class_pred_source, y_source)
    discrim_loss = get_discriminator_loss(domain_pred_source, domain_pred_target)

    feature_optim.zero_grad()

    # Update discriminator
    discriminator_optim.zero_grad()
    discrim_loss.backward(retain_graph=True)
    discriminator_optim.step()

    # Update classifier
    class_optim.zero_grad()
    class_loss.backward(retain_graph=True)
    class_optim.step()

    # Update feature extractor
    feature_optim.step()  

    # Total loss
    total_loss = class_loss - domain_adapt * discrim_loss 

    if batch % 10 == 0:
      class_loss, discrim_loss, current = class_loss.item(), discrim_loss.item(), batch * len(X_source)
      total_loss = total_loss.item()
      # print(f"## Meter  ## [{current:>5d}/{size:>5d}]")
      print(f"## Meter  ## classification loss: {class_loss:>7f} discrim loss: {discrim_loss:>7f} total loss: {total_loss:>7f}[{current:>5d}/{size:>5d}]")

    del class_loss, discrim_loss 
    del X_source, y_source, X_target, class_pred_source, domain_pred_source, domain_pred_target
  
  # return best_state, best_loss

### 4.4 Adversarial Test Loop

In [34]:
def adversarial_test_loop(dataloader, model, device, name=""):
  test_loss, correct = 0, 0

  class_loss_func = get_class_loss_func()

  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      class_pred, _ = model(X)

      test_loss += class_loss_func(class_pred, y).item()
      correct += (class_pred.argmax(1) == y).type(torch.float).sum().item()

  size = len(dataloader.dataset)
  num_batches = len(dataloader)

  test_loss /= num_batches
  correct /= size
  print(f"{name} Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

  return test_loss, correct

### 4.5 Adversarial Training

In [35]:
def adversarial_training(model, source_loader, source_test_loader, target_loader, config, device):
  # print(f"Learning_rate {config['lr']}, weight_decay {config['wd']}")
  # best_loss, best_state = float('inf'), None
  no_improve_count = 0

  for epoch in range(config['epochs']):
    print(f"Epoch {epoch+1}\n------------------")
    progress = epoch/config['epochs']

    adversarial_train_loop(source_loader, target_loader, model, config, progress, device)

    source_loss, _ = adversarial_test_loop(source_test_loader, model, device, "Source Test")

  print("Done")

## 5 Training with UDA Techniques

In [36]:
torch.cuda.empty_cache()

In [37]:
adv_model = DANN(len(product_dataset.classes)).to(device)

### 5.1 Product -> Real Life

#### 5.1.1 Training on Product

In [38]:
train_dataloader, train_test_dataloader = get_dataloader(product_dataset, config['batch_size'])
target_dataloader, target_test_dataloader = get_dataloader(real_life_dataset, config['batch_size'])

In [39]:
torch.autograd.set_detect_anomaly(True)
adversarial_training(adv_model, train_dataloader, train_test_dataloader, target_dataloader, config, device)

Epoch 1
------------------
## Meter  ## classification loss: 3.012991 discrim loss: 0.696010 total loss: 3.012991[    0/ 1601]
## Meter  ## classification loss: 2.301223 discrim loss: 0.694513 total loss: 2.301223[  640/ 1601]
## Meter  ## classification loss: 0.831948 discrim loss: 0.695082 total loss: 0.831948[ 1280/ 1601]
Source Test Test Error: 
 Accuracy: 83.0%, Avg loss: 0.575779 

Epoch 2
------------------
## Meter  ## classification loss: 0.487271 discrim loss: 0.696588 total loss: 0.165366[    0/ 1601]
## Meter  ## classification loss: 0.192116 discrim loss: 0.695409 total loss: -0.129245[  640/ 1601]
## Meter  ## classification loss: 0.479213 discrim loss: 0.694410 total loss: 0.158314[ 1280/ 1601]
Source Test Test Error: 
 Accuracy: 87.7%, Avg loss: 0.391316 

Epoch 3
------------------
## Meter  ## classification loss: 0.334134 discrim loss: 0.694381 total loss: -0.194703[    0/ 1601]
## Meter  ## classification loss: 0.153935 discrim loss: 0.694555 total loss: -0.375034[ 

#### 5.1.2 Testing on Real Life

In [40]:
loader_target_dataset = DataLoader(real_life_dataset, batch_size=config['batch_size'], shuffle=False)

test_loop(loader_target_dataset, adv_model, device)

Test Error: 
 Accuracy: 62.5%, Avg loss: 1.264015 



(1.264014733955264, 0.625)

### 5.2 Real Life -> Product

#### 5.2.1 Training on Real Life

In [41]:
del adv_model

In [42]:
train_dataloader, train_test_dataloader = get_dataloader(real_life_dataset, config['batch_size'])
target_dataloader, target_test_dataloader = get_dataloader(product_dataset, config['batch_size'])

In [43]:
# Training
adv_model = DANN(len(product_dataset.classes)).to(device)
adversarial_training(adv_model, train_dataloader, train_test_dataloader, target_dataloader, config, device)

Epoch 1
------------------
## Meter  ## classification loss: 3.008960 discrim loss: 0.702276 total loss: 3.008960[    0/ 1601]
## Meter  ## classification loss: 2.760247 discrim loss: 0.696447 total loss: 2.760247[  640/ 1601]
## Meter  ## classification loss: 2.114810 discrim loss: 0.694701 total loss: 2.114810[ 1280/ 1601]
Source Test Test Error: 
 Accuracy: 68.4%, Avg loss: 1.152067 

Epoch 2
------------------
## Meter  ## classification loss: 0.946318 discrim loss: 0.695270 total loss: 0.625022[    0/ 1601]
## Meter  ## classification loss: 0.764657 discrim loss: 0.696100 total loss: 0.442977[  640/ 1601]
## Meter  ## classification loss: 0.667452 discrim loss: 0.695120 total loss: 0.346225[ 1280/ 1601]
Source Test Test Error: 
 Accuracy: 74.4%, Avg loss: 0.799938 

Epoch 3
------------------
## Meter  ## classification loss: 0.548629 discrim loss: 0.695192 total loss: 0.019175[    0/ 1601]
## Meter  ## classification loss: 0.486324 discrim loss: 0.695851 total loss: -0.043632[  6

#### 5.2.2 Testing on Product

In [44]:
loader_target_dataset = DataLoader(product_dataset, batch_size=config['batch_size'], shuffle=False)

test_loop(loader_target_dataset, adv_model, device)

Test Error: 
 Accuracy: 79.9%, Avg loss: 0.693335 



(0.6933353942586109, 0.799)

In [45]:
# del source_dataset, train_dataloader, test_dataloader, target_dataset, loader_target_dataset
del adv_model

## 6 UDA Ablation Study

### 6.1 How Different Loss Functions Work

### 6.2 How UDA over different levels of feature layer work

In [47]:
# To test the middle layer feature adversarial training, we copy the model from torchvision and modify it.
class AlexNet(nn.Module):
    def __init__(self, num_classes: int = 1000, dropout: float = 0.5) -> None:
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )# 12544 = (6*6*256)
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(256 * 6 * 6, 4096), 
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),  # original 4096
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        feature = self.features(x)
        x = self.avgpool(feature)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        feature = feature.view(feature.size(0), -1)
        # return x, x
        return x, feature

In [48]:
class FeatureExtractor(nn.Module):
  def __init__(self, pretrained=True):
    super(FeatureExtractor, self).__init__()
    state_dict = alexnet(pretrained=pretrained).state_dict()
    self.feature_extractor = AlexNet()
    self.feature_extractor.load_state_dict(state_dict)
    self.feature_dim = self.feature_extractor.classifier[-1].in_features
    self.adv_feature_dim = 12544
    print(self.feature_dim, self.adv_feature_dim)
    # print(f"Feature dimension: {self.feature_dim}")
    # make the last layer identity
    self.feature_extractor.classifier[-1] = nn.Identity()

  def forward(self, x):
    out = self.feature_extractor(x)
    return out
  
  def output_dim(self):
    return self.feature_dim
  
  def adv_output_dim(self):
    return self.adv_feature_dim

In [49]:
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.discriminator =  nn.Sequential(
            nn.Linear(int(input_dim), 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024,1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Sigmoid()
        )

    def forward(self, x):
        validity = self.discriminator(x)
        return validity 

In [50]:
class DANN(nn.Module):
  # def __init__(self, num_classes, adversarial=True):
  def __init__(self, num_classes, pretrained=True):
    super(DANN, self).__init__()
    self.output_dim = num_classes

    # define inner network component
    self.feature_extractor = FeatureExtractor(pretrained=pretrained)
    self.classifier = Classifier(self.feature_extractor.output_dim(), num_classes)
    self.discriminator = Discriminator(self.feature_extractor.adv_output_dim())  
  
  def forward(self, x):
    # 4096, 12544
    feature_output, adv_feature = self.feature_extractor(x)
    
    class_pred = self.classifier(feature_output)

    # Add a ReverseLayer here for negative gradient computation
    reverse_feature = ReverseLayerF.apply(adv_feature)
    domain_pred = self.discriminator(reverse_feature)

    return class_pred, domain_pred 

Product -> Real Life

In [51]:
train_dataloader, train_test_dataloader = get_dataloader(product_dataset, config['batch_size'])
target_dataloader, target_test_dataloader = get_dataloader(real_life_dataset, config['batch_size'])
adv_model = DANN(len(product_dataset.classes), pretrained=True).to(device)
torch.autograd.set_detect_anomaly(True)
adversarial_training(adv_model, train_dataloader, train_test_dataloader, target_dataloader, config, device)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


4096 12544
Epoch 1
------------------
## Meter  ## classification loss: 3.007089 discrim loss: 0.703104 total loss: 3.007089[    0/ 1601]
## Meter  ## classification loss: 2.242389 discrim loss: 0.698007 total loss: 2.242389[  640/ 1601]
## Meter  ## classification loss: 0.674071 discrim loss: 0.696746 total loss: 0.674071[ 1280/ 1601]
Source Test Test Error: 
 Accuracy: 82.5%, Avg loss: 0.701569 

Epoch 2
------------------
## Meter  ## classification loss: 0.737465 discrim loss: 0.697060 total loss: 0.415342[    0/ 1601]
## Meter  ## classification loss: 0.268852 discrim loss: 0.695527 total loss: -0.052563[  640/ 1601]
## Meter  ## classification loss: 0.298298 discrim loss: 0.695144 total loss: -0.022940[ 1280/ 1601]
Source Test Test Error: 
 Accuracy: 87.0%, Avg loss: 0.544255 

Epoch 3
------------------
## Meter  ## classification loss: 0.321255 discrim loss: 0.694437 total loss: -0.207624[    0/ 1601]
## Meter  ## classification loss: 0.227817 discrim loss: 0.694351 total loss:

In [52]:
loader_target_dataset = DataLoader(real_life_dataset, batch_size=config['batch_size'], shuffle=False)

test_loop(loader_target_dataset, adv_model, device)

Test Error: 
 Accuracy: 63.8%, Avg loss: 1.271149 



(1.2711493540555239, 0.6385)

Real Life -> Product

In [53]:
config = dict(epochs=10, batch_size=64, lr=0.01, wd=0.001, momentum=0.9, alpha=10, beta=0.75, gamma=10)
train_dataloader, train_test_dataloader = get_dataloader(real_life_dataset, config['batch_size'])
target_dataloader, target_test_dataloader = get_dataloader(product_dataset, config['batch_size'])
adv_model = DANN(len(real_life_dataset.classes), pretrained=True).to(device)
torch.autograd.set_detect_anomaly(True)
adversarial_training(adv_model, train_dataloader, train_test_dataloader, target_dataloader, config, device)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


4096 12544
Epoch 1
------------------
## Meter  ## classification loss: 2.998693 discrim loss: 0.724238 total loss: 2.998693[    0/ 1601]
## Meter  ## classification loss: 2.731299 discrim loss: 0.710308 total loss: 2.731299[  640/ 1601]
## Meter  ## classification loss: 1.763867 discrim loss: 0.695495 total loss: 1.763867[ 1280/ 1601]
Source Test Test Error: 
 Accuracy: 66.4%, Avg loss: 1.300384 

Epoch 2
------------------
## Meter  ## classification loss: 0.989391 discrim loss: 0.697166 total loss: 0.667219[    0/ 1601]
## Meter  ## classification loss: 0.933943 discrim loss: 0.694500 total loss: 0.613003[  640/ 1601]
## Meter  ## classification loss: 1.072262 discrim loss: 0.695066 total loss: 0.751060[ 1280/ 1601]
Source Test Test Error: 
 Accuracy: 74.2%, Avg loss: 0.829410 

Epoch 3
------------------
## Meter  ## classification loss: 0.645482 discrim loss: 0.694187 total loss: 0.116793[    0/ 1601]
## Meter  ## classification loss: 0.559664 discrim loss: 0.694190 total loss: 0.

In [54]:
loader_target_dataset = DataLoader(product_dataset, batch_size=config['batch_size'], shuffle=False)

test_loop(loader_target_dataset, adv_model, device)

Test Error: 
 Accuracy: 79.6%, Avg loss: 0.696960 



(0.6969601637683809, 0.796)

## 7 Summary

### 6.1 Product -> Real Life


#### 6.1.1 Purely training on Source Domain-Product

Without using any domain adaptation techniques, within 10 epochs training, the classifier network achieves  64.6% accuracy on the target domain.

#### 6.1.2 Training on both Source and Target Domains

Using DANN doamin adaptation technique, the classifier with feature extractor trained on both source and target domain achives 64.6% accuracy, which is the same as the one trained solely on the source domain, with no improvement on accuracy.

### 6.2 Real Life -> Product

#### 6.2.1 Purely Training on Source Domain-Real Life

The classifier trained solely on the source domain achives 79.2% accuracy on the target domain.

#### 6.2.2 Training on both Source and Target Domains

With feature extractor trained on both source and target domain, the classifer achives 91.9% accuracy on the target domain, which is about 12% improvement over feature extractor purely trained on source domain.